In [1]:
# Uncomment the line below and run if you are missing the rasterio package. 
# You might need to restart your kernel after for it to work

# %pip install rasterio
# %pip install scikit-image

In [2]:
import tensorflow as tf
import numpy as np
import rasterio
import os
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from preprocess.wildfire_preprocess import create_sequences

2025-02-12 16:25:32.970442: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-12 16:25:33.010924: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9373] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-12 16:25:33.010955: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-12 16:25:33.012203: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1534] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-12 16:25:33.018660: I tensorflow/core/platform/cpu_feature_guar

In [3]:
# Open the TIFF file
with rasterio.open('/data/ai_club/fire/WildfireSpreadLS/2018/fire_21889697/2018-08-25.tif') as src:
    print(f"Number of bands: {src.count}")
    
    # Print metadata for each band
    for i in range(1, src.count + 1):
        description = src.descriptions[i - 1]  # Band descriptions (if available)
        band_data = src.read(i)  # Read band i
        print(f"Band {description}: min={band_data.min()}, max={band_data.max()}")

Number of bands: 23
Band M11: min=85.0, max=8129.0
Band I2: min=439.0, max=5024.0
Band I1: min=-100.0, max=3849.0
Band NDVI_last: min=nan, max=nan
Band EVI2_last: min=nan, max=nan
Band total precipitation: min=nan, max=nan
Band wind speed: min=nan, max=nan
Band wind direction: min=nan, max=nan
Band minimum temperature: min=nan, max=nan
Band maximum temperature: min=nan, max=nan
Band energy release component: min=nan, max=nan
Band specific humidity: min=nan, max=nan
Band slope: min=nan, max=nan
Band aspect: min=nan, max=nan
Band elevation: min=nan, max=nan
Band pdsi: min=nan, max=nan
Band LC_Type1: min=1.0, max=17.0
Band total_precipitation_surface_sum: min=0.0, max=3.375
Band forecast wind speed: min=0.6297666430473328, max=2.8383054733276367
Band forecast wind direction: min=-86.51997375488281, max=88.85616302490234
Band forecast temperature: min=14.965868949890137, max=18.317867279052734
Band forecast specific humidity: min=0.007220204453915358, max=0.008270804770290852
Band active f

In [4]:
fire_name = '2018/fire_21889697' # The fire with the most files

In [5]:
# Directory where the TIFF files are stored
data_dir = f"/data/ai_club/fire/WildfireSpreadLS/{fire_name}"  # Replace with the actual path if needed

# Get the list of all .tif files in the directory
file_paths = sorted([os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith(".tif")])

# Ensure the files are sorted by date to maintain temporal order
print(f"Total files found: {len(file_paths)}")

# Create sequences and labels
sequence_length = 3  # Length of input sequence
train_sequences, train_labels = create_sequences(file_paths, sequence_length=sequence_length)

# Split data into training and validation sets (80% training, 20% validation)
train_sequences, val_sequences, train_labels, val_labels = train_test_split(
    train_sequences, train_labels, test_size=0.2, random_state=42
)

train_labels = np.expand_dims(train_labels, axis=-1)  # Add channel dimension: (N, H, W, 1)
val_labels = np.expand_dims(val_labels, axis=-1)  # Add channel dimension: (N, H, W, 1)

# Verify shapes before training
print("Train Sequences Shape:", train_sequences.shape)  # Should be (N, 3, 300, 220, 23)
print("Train Labels Shape:", train_labels.shape)       # Should be (N, 300, 220, 1)
print("Val Sequences Shape:", val_sequences.shape)     # Should be (N, 3, 300, 220, 23)
print("Val Labels Shape:", val_labels.shape)   

Total files found: 94


/home/balbonis/fire-research/wildfire_preprocess.py:63: RuntimeWarning: Mean of empty slice.
  band = np.nan_to_num(band, nan=band[~np.isnan(band)].mean())
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)


Train Sequences Shape: (67, 3, 300, 220, 23)
Train Labels Shape: (67, 300, 220, 1)
Val Sequences Shape: (17, 3, 300, 220, 23)
Val Labels Shape: (17, 300, 220, 1)


In [6]:
def build_unet(input_shape):
    inputs = tf.keras.Input(shape=input_shape)
    
    # Split inputs by feature groups for specialized processing
    # Indices based on the dataset description
    viirs_features = inputs[..., :3]  # VIIRS bands (I1, I2, M11)
    weather_features = inputs[..., 3:13]  # GRIDMET features
    forecast_features = inputs[..., 13:18]  # GFS features
    static_features = inputs[..., 18:]  # Land cover, elevation, etc.
    
    # Encoder with separate paths for different feature types
    def encoder_block(x, filters, name):
        x = tf.keras.layers.Conv3D(filters, (3, 3, 3), activation='relu', padding='same', name=f'{name}_conv1')(x)
        x = tf.keras.layers.BatchNormalization(name=f'{name}_bn1')(x)
        x = tf.keras.layers.Conv3D(filters, (3, 3, 3), activation='relu', padding='same', name=f'{name}_conv2')(x)
        x = tf.keras.layers.BatchNormalization(name=f'{name}_bn2')(x)
        skip = x
        x = tf.keras.layers.MaxPooling3D((1, 2, 2), name=f'{name}_pool')(x)
        return x, skip
    
    # Process different feature groups
    viirs_enc1, viirs_skip1 = encoder_block(viirs_features, 32, 'viirs1')
    viirs_enc2, viirs_skip2 = encoder_block(viirs_enc1, 64, 'viirs2')
    
    weather_enc1, weather_skip1 = encoder_block(weather_features, 32, 'weather1')
    weather_enc2, weather_skip2 = encoder_block(weather_enc1, 64, 'weather2')
    
    # Combine features
    combined = tf.keras.layers.Concatenate()([viirs_enc2, weather_enc2])
    
    # Bridge
    bridge = tf.keras.layers.Conv3D(128, (3, 3, 3), activation='relu', padding='same')(combined)
    bridge = tf.keras.layers.BatchNormalization()(bridge)
    bridge = tf.keras.layers.SpatialDropout3D(0.3)(bridge)  # Add dropout to prevent overfitting
    
    # Decoder with attention
    def attention_block(x, skip_connection):
        g1 = tf.keras.layers.Conv3D(x.shape[-1], (1, 1, 1))(skip_connection)
        g1 = tf.keras.layers.BatchNormalization()(g1)
        x1 = tf.keras.layers.Conv3D(x.shape[-1], (1, 1, 1))(x)
        x1 = tf.keras.layers.BatchNormalization()(x1)
        psi = tf.keras.layers.Activation('relu')(g1 + x1)
        psi = tf.keras.layers.Conv3D(1, (1, 1, 1))(psi)
        psi = tf.keras.layers.Activation('sigmoid')(psi)
        return skip_connection * psi
    
    # Decoder
    def decoder_block(x, skip, filters):
        x = tf.keras.layers.Conv3DTranspose(filters, (3, 3, 3), strides=(1, 2, 2), padding='same')(x)
        attention = attention_block(x, skip)
        x = tf.keras.layers.Concatenate()([x, attention])
        x = tf.keras.layers.Conv3D(filters, (3, 3, 3), activation='relu', padding='same')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        return x
    
    # Decoder path
    dec1 = decoder_block(bridge, tf.keras.layers.Concatenate()([viirs_skip2, weather_skip2]), 64)
    dec2 = decoder_block(dec1, tf.keras.layers.Concatenate()([viirs_skip1, weather_skip1]), 32)
    
    # Final convolution with class balancing
    outputs = tf.keras.layers.Conv3D(1, (3, 1, 1), padding='valid')(dec2)
    outputs = tf.keras.layers.Reshape((300, 220, 1))(outputs)
    outputs = tf.keras.layers.Activation('sigmoid')(outputs)
    
    model = tf.keras.Model(inputs, outputs)
    
    # Custom weighted loss combining focal loss and IoU loss
    def combined_loss(y_true, y_pred):
        # Focal loss component
        alpha = 0.75  # Give more weight to fire pixels since they're rare
        gamma = 2.0
        epsilon = tf.keras.backend.epsilon()
        y_pred = tf.clip_by_value(y_pred, epsilon, 1 - epsilon)
        focal = -alpha * y_true * tf.pow(1 - y_pred, gamma) * tf.math.log(y_pred) - \
                (1 - alpha) * (1 - y_true) * tf.pow(y_pred, gamma) * tf.math.log(1 - y_pred)
        
        # IoU loss component
        intersection = tf.reduce_sum(y_true * y_pred)
        union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) - intersection
        iou = (intersection + epsilon) / (union + epsilon)
        iou_loss = 1 - iou
        
        # Combine losses
        return tf.reduce_mean(focal) + iou_loss
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4, clipnorm=1.0),
        loss=combined_loss,
        metrics=['accuracy', iou_metric]
    )
    
    return model

In [9]:
def iou_metric(y_true, y_pred):
    y_pred = tf.round(y_pred)  # Round predictions to 0 or 1
    intersection = tf.reduce_sum(y_true * y_pred)
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) - intersection
    iou = (intersection + tf.keras.backend.epsilon()) / (union + tf.keras.backend.epsilon())
    return iou

In [10]:
# Calculate class weights based on dataset statistics
# From the paper, we know there's significant class imbalance
pos_weight = np.sum(train_labels == 0) / (np.sum(train_labels == 1) + 1e-6)
class_weight = {0: 1.0, 1: pos_weight}

# Add more sophisticated callbacks
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_iou_metric',
        patience=10,
        mode='max',
        restore_best_weights=True
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_iou_metric',
        factor=0.5,
        patience=5,
        mode='max',
        min_lr=1e-6
    ),
    tf.keras.callbacks.ModelCheckpoint(
        'best_model.h5',
        monitor='val_iou_metric',
        mode='max',
        save_best_only=True
    ),
    # Custom learning rate schedule
    tf.keras.callbacks.LearningRateScheduler(
        lambda epoch: 1e-4 * (0.95 ** epoch)
    )
]

# Define the input shape based on your data
input_shape = (3, 300, 220, 23)

# Build the model using the defined input shape
model = build_unet(input_shape)

# Train with a larger batch size if memory allows
history = model.fit(
    train_sequences,
    train_labels,
    validation_data=(val_sequences, val_labels),
    epochs=100,
    batch_size=8,  # Increase if memory allows
    class_weight=class_weight,
    callbacks=callbacks,
    shuffle=True
)

Epoch 1/100


2025-02-12 16:31:01.483451: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:467] Loaded cuDNN version 90100
2025-02-12 16:31:24.304499: I external/local_xla/xla/service/service.cc:168] XLA service 0x7fad72ca1270 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-02-12 16:31:24.304534: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2025-02-12 16:31:24.304544: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
2025-02-12 16:31:24.304549: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (2): Tesla T4, Compute Capability 7.5
2025-02-12 16:31:24.304555: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (3): Tesla T4, Compute Capability 7.5
2025-02-12 16:31:24.308985: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set en

9/9 [==============================] - 123s 8s/step - loss: nan - accuracy: 0.9941 - iou_metric: nan - val_loss: nan - val_accuracy: 0.9944 - val_iou_metric: nan - lr: 1.0000e-04
Epoch 2/100
9/9 [==============================] - 18s 2s/step - loss: nan - accuracy: 0.9941 - iou_metric: nan - val_loss: nan - val_accuracy: 0.9944 - val_iou_metric: nan - lr: 9.5000e-05
Epoch 3/100
9/9 [==============================] - 13s 1s/step - loss: nan - accuracy: 0.9941 - iou_metric: nan - val_loss: nan - val_accuracy: 0.9944 - val_iou_metric: nan - lr: 9.0250e-05
Epoch 4/100
3/9 [=========>....................] - ETA: 8s - loss: nan - accuracy: 0.9950 - iou_metric: nan 


KeyboardInterrupt



In [ ]:
# Plot training history
def plot_training_history(history):
    """Plot training and validation metrics"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Plot loss
    ax1.plot(history.history['loss'], label='Training Loss')
    ax1.plot(history.history['val_loss'], label='Validation Loss')
    ax1.set_title('Model Loss')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.legend()
    
    # Plot IoU metric
    ax2.plot(history.history['iou_metric'], label='Training IoU')
    ax2.plot(history.history['val_iou_metric'], label='Validation IoU')
    ax2.set_title('Model IoU')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('IoU')
    ax2.legend()
    
    plt.tight_layout()
    plt.show()

# Evaluate model and show detailed metrics
def evaluate_model(model, val_sequences, val_labels):
    """Evaluate model and print detailed metrics"""
    # Get predictions
    predictions = model.predict(val_sequences)
    predictions_binary = (predictions > 0.5).astype(np.float32)
    
    # Calculate metrics
    val_loss, val_acc, val_iou = model.evaluate(val_sequences, val_labels, verbose=0)
    
    # Calculate additional metrics
    tp = np.sum(predictions_binary * val_labels)
    fp = np.sum(predictions_binary * (1 - val_labels))
    fn = np.sum((1 - predictions_binary) * val_labels)
    tn = np.sum((1 - predictions_binary) * (1 - val_labels))
    
    precision = tp / (tp + fp + 1e-7)
    recall = tp / (tp + fn + 1e-7)
    f1 = 2 * (precision * recall) / (precision + recall + 1e-7)
    
    print("\nModel Evaluation Metrics:")
    print(f"{'Metric':<20} {'Value':<10}")
    print("-" * 30)
    print(f"{'Loss':<20} {val_loss:.4f}")
    print(f"{'Accuracy':<20} {val_acc:.4f}")
    print(f"{'IoU Score':<20} {val_iou:.4f}")
    print(f"{'Precision':<20} {precision:.4f}")
    print(f"{'Recall':<20} {recall:.4f}")
    print(f"{'F1 Score':<20} {f1:.4f}")
    
    # Print confusion matrix
    print("\nConfusion Matrix:")
    print(f"{'':>10}{'Predicted':>20}")
    print(f"{'Actual':>10}{'No Fire':>10}{'Fire':>10}")
    print(f"{'No Fire':>10}{tn:>10.0f}{fp:>10.0f}")
    print(f"{'Fire':>10}{fn:>10.0f}{tp:>10.0f}")
    
    return predictions_binary

# Visualize example predictions
def plot_example_predictions(val_sequences, val_labels, predictions, num_examples=3):
    """Plot example predictions vs ground truth"""
    fig, axes = plt.subplots(num_examples, 3, figsize=(15, 5*num_examples))
    
    for i in range(num_examples):
        # Input sequence (last frame)
        axes[i, 0].imshow(val_sequences[i, -1, :, :, -1])
        axes[i, 0].set_title(f'Input (Last Frame)')
        axes[i, 0].axis('off')
        
        # Ground truth
        axes[i, 1].imshow(val_labels[i])
        axes[i, 1].set_title('Ground Truth')
        axes[i, 1].axis('off')
        
        # Prediction
        axes[i, 2].imshow(predictions[i])
        axes[i, 2].set_title('Prediction')
        axes[i, 2].axis('off')
    
    plt.tight_layout()
    plt.show()

# After training the model
plot_training_history(history)

# Evaluate the model
predictions = evaluate_model(model, val_sequences, val_labels)

# Visualize some example predictions
plot_example_predictions(val_sequences, val_labels, predictions)